In [ ]:
library(stringr) 
library(dplyr) 
library(readr) 
library(gower)
library(ipred)
library(caret) 
library(MASS) 
library(nnet)  
library(kernlab) 
library(class) 
library(KernelKnn) 
library(nnet) 
library(e1071) 
library(gbm)
library(xgboost) 
library(glmnet)
library(caret)
library(plotmo)
library(ggplot2)
library(randomForest)
library(factoextra)


In [ ]:
#1.Combine Dataset(need information of PAM50)
data1 = read_csv("C:/Users/36961/Desktop/breastcancerproteomes/77_cancer_proteomes_CPTAC_itraq.csv")
dim(data1)
rownames(data1)
data1
data1$`Complete TCGA ID`



data2 = read_csv("C:/Users/36961/Desktop/breastcancerproteomes/clinical_data_breast_cancer.csv")
dim(data2)
colnames(data2)
data2link = data2[,c(1,21)]

for (i in 1:83){
  a1 = str_replace(data1$'Complete TCGA ID', "TCGA", "")
  a2 = str_replace(a1, ".\\d+$","")
  a3 = str_replace(a2,"^","TCGA-")
  data1$`Complete TCGA ID`[i]<-a3[i]
}

combined_data = inner_join(data1,data2link)
dim(combined_data)
combined_data


In [ ]:
#2. Dealing with missing value
percent = function(x) {
  sum(!is.na(x))/length(x)
}
percent_new = sapply(combined_data, percent)
cleaningset = combined_data[1]
for (i in 2:12554) {
  if(percent_new[i]>0.99) {
    cleaningset = cbind(cleaningset, combined_data[i])
  }
}
cleaningset[is.na(cleaningset)] = 0
dataset<-cleaningset

dataset<-dataset[-1]
attach(dataset)
dim(dataset)


In [ ]:
#2.1 Visualization the response of combined dataset.
dt = data.frame(
  group = c("Basal_like", "HER2_enriched", "Luminal_A", "Luminal_B"),
  value = c(19, 13, 23,25)
)
dt = dt[order(dt$value, decreasing = TRUE),]
myLabel = as.vector(dt$group)   
myLabel = paste(myLabel, "(", round(dt$value / sum(dt$value) * 100, 2), "%)", sep = "")   

p = ggplot(dt, aes(x = "", y = value, fill = group)) +
  geom_bar(stat = "identity", width = 1) +    
  coord_polar(theta = "y") + 
  labs(x = "", y = "", title = "") + 
  theme(axis.ticks = element_blank()) + 
  theme(legend.title = element_blank(), legend.position = "top") + 
  scale_fill_discrete(breaks = dt$group, labels = myLabel) + 
  theme(axis.text.x = element_blank()) + 
  geom_text(aes(y = value/2 + c(0, cumsum(value)[-length(value)]), x = sum(value)/80, label = myLabel), size = 3)
p


In [ ]:
#3. Using LASSO to do variable selection
dataset$`PAM50 mRNA`<- as.factor(dataset$`PAM50 mRNA`)
x = model.matrix(dataset$`PAM50 mRNA`~., dataset)
train = sample(1:nrow(x), 3*nrow(x)/4)
test = (-train)
y = dataset$`PAM50 mRNA`
test_y = y[test]
train_y = y[train]
test_x = x[test,]
train_x = x[train,]
set.seed(1) 

cvfit = cv.glmnet(train_x,train_y, alpha=1, family="multinomial",type.multinomial="grouped")
mod_lasso = glmnet(train_x,train_y,alpha=1, family="multinomial",type.multinomial="grouped", lambda = cvfit$lambda)
plot(cvfit)
plot_glmnet(mod_lasso,nresponse = 4)

mod_lasso = glmnet(train_x,train_y,alpha=1, family="multinomial",type.multinomial="grouped", lambda = cvfit$lambda.min)
lassocoef = predict(mod_lasso, type ="coefficients",s=cvfit$lambda.min,family="multinomial",type.multinomial="grouped")
sink(file="lassomod.csv")
options("max.print"=8020)
lassocoef
sink(NULL)



In [ ]:
#4. Set up the training and test set under dimension reduction. (61)
testdataset = dataset[,c(8018,141,342,774,1060,1069,1164,1280,1281,1376,1377,1378,
                         1474,1570,1716,1717,1848,1916,2037,2117,2403,2440,2570,2634,
                         2652,3438,3609,3629,3889,4088,4124,4125,4211,4278,4536,4550,
                         4657,4816,4925,5002,5218,5714,5784,5819,5826,5900,5958,6333,
                         6662,6769,6873,7053,7357,7378,7425,7512,7515,7597,7827,7836,7841,
                         7924)]
testdataset$`PAM50 mRNA`<- as.factor(testdataset$`PAM50 mRNA`)
dim(testdataset)



In [ ]:
#5. Multinomial Method
acc_m = c(0,0,0)
for(i in 1:1000){
  train = createDataPartition(testdataset$`PAM50 mRNA`, p = .75, list = FALSE)
  trainset = testdataset[train,]
  testset = testdataset[-train,]
  multinomial_method = multinom(`PAM50 mRNA`~., data=trainset,trace=FALSE)
  acc_m[i] = confusionMatrix(predict(multinomial_method, newdata=testset, "class"),testset$`PAM50 mRNA`)$overall["Accuracy"]
}
plot(density(acc_m))
summary(acc_m)


In [ ]:
#6. Neural Network
acc_neural = c(0,0,0)
for(i in 1:1000){
  train = createDataPartition(testdataset$`PAM50 mRNA`, p = .75, list = FALSE)
  trainset = testdataset[train,]
  testset = testdataset[-train,]
  trainset$`PAM50 mRNA`<- as.factor(trainset$`PAM50 mRNA`)
  neural_method = train(`PAM50 mRNA` ~., data=trainset, method = "nnet",preProc = c("center", "scale"), trace = FALSE)
  neural_pred = predict(neural_method, newdata=testset)
  acc_neural[i] = confusionMatrix(testset$`PAM50 mRNA`,neural_pred)$overall["Accuracy"]
}
plot(density(acc_neural))
summary(acc_neural)


In [ ]:
#7. knn
acc_knn = c(0,0,0)
for (i in 1:1000){
  knn_method = knn.cv(knn_x, knn_y, k = 3, prob= TRUE)
  acc_knn[i] = confusionMatrix(knn_method,testdataset$`PAM50 mRNA`)$overall["Accuracy"]
}
summary(acc_knn)
plot(density(acc_knn))


In [ ]:
#8. svm
acc_svm = c(0,0,0)
for(i in 1:1000){
  train_svm = createDataPartition(testdataset$`PAM50 mRNA`, p = .75, list = FALSE)
  trainset_svm = testdataset[train_svm,]
  testset_svm = testdataset[-train_svm,]
  svm_method = ksvm(`PAM50 mRNA` ~., scale = TRUE, data=trainset_svm,
                kernel="rbfdot", prob.model=TRUE)
  acc_svm[i] = confusionMatrix(testset_svm$`PAM50 mRNA`, predict(svm_method, testset_svm))$overall["Accuracy"]
}
summary(acc_svm)
plot(density(acc_svm))


In [ ]:
#9. Naive Bayes
acc_nb = c(0.00)
for (i in 1:1000) {
  train_nb = createDataPartition(testdataset$`PAM50 mRNA`, p = .75, list = FALSE)
  trainset_nb = testdataset[train_nb,]
  testset_nb = testdataset[-train_nb,]
  nb_method = naiveBayes(`PAM50 mRNA`~.,data = trainset_nb) 
  pre_nb = predict(nb_method, newdata=testset_nb)
  acc_nb[i] = confusionMatrix(testset_nb$`PAM50 mRNA`,pre_nb)$overall["Accuracy"]
}
summary(acc_nb)
plot(density(acc_nb))
